In [16]:
import pandas as pd

# Read a sample of the data
html = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet'
df_green = pd.read_parquet(html)

# Display first rows
df_green.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


In [63]:
import pandas as pd

# Read a sample of the data
html = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv'
df_zone = pd.read_csv(html)

# Display first rows
df_zone.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [22]:
start = pd.Timestamp("2025-11-01")
end   = pd.Timestamp("2025-12-01")

filter = (
    (df_green["lpep_pickup_datetime"] >= start) &
    (df_green["lpep_pickup_datetime"] < end) &
    (df_green["trip_distance"] <= 1.0)
)

print(f'Short trip count: {filter.sum()}')


Short trip count: 8007


In [37]:
filter = (df_green["trip_distance"] < 100)
pu_date = df_green[filter].sort_values("trip_distance",ascending=False).head(1)

print(pu_date["lpep_pickup_datetime"].iloc[0].date())


2025-11-14


In [87]:
filter = (df_green["lpep_pickup_datetime"] >= pd.Timestamp("2025-11-01"))

top_zone = (df_green.loc[filter]
            .groupby("PULocationID")["total_amount"]
            .sum()
            .sort_values(ascending=False)
            .head(1))

pu_name = pd.merge(left=top_zone,
                     right=df_zone,
                     how = "left",
                     left_on="PULocationID",
                     right_on="LocationID").iloc[0]

print(pu_name["Zone"])

East Harlem North


In [88]:
start = pd.Timestamp("2025-11-01")
end   = pd.Timestamp("2025-12-01")
ZoneID = (df_zone[(df_zone["Zone"] == "East Harlem North")].LocationID.iloc[0])

filter = (
    (df_green["lpep_pickup_datetime"] >= start) &
    (df_green["lpep_pickup_datetime"] < end) &
    (df_green["PULocationID"] == ZoneID)
)

top_tip = (df_green.loc[filter]
            .groupby("DOLocationID")["tip_amount"]
            .max()
            .idxmax()
        )

do_zone_name = df_zone.loc[df_zone["LocationID"] == top_tip, "Zone"].iloc[0]

print(do_zone_name)

Yorkville West
